#### 0.0. IMPORTS

In [1]:
import pandas as pd

import inflection 

import math
import numpy as np

import seaborn as sns
from matplotlib import pyplot as plt

##### 0.1 Helper Functions 

##### 0.2 Loading Data

In [2]:
df_sales_raw = pd.read_csv('data/train.csv', low_memory=False)
df_store_raw = pd.read_csv('data/store.csv', low_memory=False)

#merge
df_raw = pd.merge(df_sales_raw, df_store_raw, how='left', on='Store')
df_raw.head(3)

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,5,2015-07-31,6064,625,1,1,0,1,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,5,2015-07-31,8314,821,1,1,0,1,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"


#### 1.0 DESCRIÇÃO DOS DADOS

In [3]:
df1 = df_raw.copy()

##### 1.1. Rename Columns

In [4]:
df_raw.columns

Index(['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
       'CompetitionDistance', 'CompetitionOpenSinceMonth',
       'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
       'Promo2SinceYear', 'PromoInterval'],
      dtype='object')

In [5]:
#função que irá formatar as palavras da coluna do dataframe. Eles ficaram todos com letra minusculas e separadas por _
cols_old = ['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo', 'StateHoliday', 
            'SchoolHoliday', 'StoreType', 'Assortment',
            'CompetitionDistance', 'CompetitionOpenSinceMonth',
            'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
            'Promo2SinceYear', 'PromoInterval']

snakecase = lambda x: inflection.underscore(x)

cols_new = list(map(snakecase,cols_old))

In [6]:
df1.columns = cols_new

In [7]:
df1.columns

Index(['store', 'day_of_week', 'date', 'sales', 'customers', 'open', 'promo',
       'state_holiday', 'school_holiday', 'store_type', 'assortment',
       'competition_distance', 'competition_open_since_month',
       'competition_open_since_year', 'promo2', 'promo2_since_week',
       'promo2_since_year', 'promo_interval'],
      dtype='object')

##### 1.2. Data Dimensions

In [8]:
print('N° total de linhas: ', df1.shape[0])
print('N° total de colunas: ', df1.shape[1])

N° total de linhas:  1017209
N° total de colunas:  18


##### 1.3 Data Types

In [9]:
# Verificação do tipo de dados
df1.dtypes

store                             int64
day_of_week                       int64
date                             object
sales                             int64
customers                         int64
open                              int64
promo                             int64
state_holiday                    object
school_holiday                    int64
store_type                       object
assortment                       object
competition_distance            float64
competition_open_since_month    float64
competition_open_since_year     float64
promo2                            int64
promo2_since_week               float64
promo2_since_year               float64
promo_interval                   object
dtype: object

In [10]:
#Convertendo a coluna date de object para datetime
df1.date = pd.to_datetime(df1.date)

In [11]:
#Verificando novamente
df1.dtypes

store                                    int64
day_of_week                              int64
date                            datetime64[ns]
sales                                    int64
customers                                int64
open                                     int64
promo                                    int64
state_holiday                           object
school_holiday                           int64
store_type                              object
assortment                              object
competition_distance                   float64
competition_open_since_month           float64
competition_open_since_year            float64
promo2                                   int64
promo2_since_week                      float64
promo2_since_year                      float64
promo_interval                          object
dtype: object

##### 1.5. Fillout NA

In [12]:
#Verificando os valores vazios
df1.isnull().sum()

store                                0
day_of_week                          0
date                                 0
sales                                0
customers                            0
open                                 0
promo                                0
state_holiday                        0
school_holiday                       0
store_type                           0
assortment                           0
competition_distance              2642
competition_open_since_month    323348
competition_open_since_year     323348
promo2                               0
promo2_since_week               508031
promo2_since_year               508031
promo_interval                  508031
dtype: int64

In [15]:
#competition_distance: 2642
#Metodo: Substituir por um valor muito alto, deixando de ser um competidor próximo
#df1.competition_distance.max()
df1.competition_distance = df1.competition_distance.apply(lambda x:200000.0 if math.isnan(x) else x)

#competition_open_since_month: 323348
#Metodo: vai ser o valor de mes da coluna Date
df1.competition_open_since_month = df1.apply(lambda x:x.date.month if math.isnan(x.competition_open_since_month) else x.competition_open_since_month, axis=1)

#competition_open_since_year: 323348
#Metodo: vai ser o valor de ano da coluna Date
df1.competition_open_since_year = df1.apply(lambda x:x.date.year if math.isnan(x.competition_open_since_year) else x.competition_open_since_year,axis=1)

#promo2_since_week: 508031
#Metodo: vai ser o valor de semana da coluna Date
df1.promo2_since_week = df1.apply(lambda x:x.date.week if math.isnan(x.promo2_since_week) else x.promo2_since_week, axis=1)

#promo2_since_year: 508031
#Metodo: vai ser o valor do ano da coluna Date
df1.promo2_since_year = df1.apply(lambda x:x.date.year if math.isnan(x.promo2_since_year) else x.promo2_since_year, axis=1)

#promo_interval: 508031
#Metodo: 1. criar um dicionario dos meses do ano, 2.preencher os vazios com valor 0, 3. , 4. 
month_map = {1:'Jan',2:'Feb',3:'Mar',4:'Apr',5:'Jun',6:'Jul',7:'Aug',8:'Sep',9:'Oct',10:'Nov',11:'Dec'}

df1.promo_interval.fillna(0, inplace=True)

df1['month_map'] = df1.date.dt.month.map(month_map)

df1['is_promo'] = df1[['promo_interval', 'month_map']].apply(lambda x:0 if x['promo_interval'] == 0 else 1 if x['month_map'] in x['promo_interval'].split(',') else 0, axis=1)

In [17]:
df1.sample(5).T

,142340,719524,896624,336287,632567
store,736,20,950,956,33
day_of_week,4,2,5,2,3
date,2015-03-26 00:00:00,2013-09-24 00:00:00,2013-04-19 00:00:00,2014-09-16 00:00:00,2013-12-11 00:00:00
sales,4515,8089,4752,4576,7542
customers,463,903,556,603,776
open,1,1,1,1,1
promo,0,1,0,1,0
state_holiday,0,0,0,0,0
school_holiday,0,0,0,0,0
store_type,c,d,a,a,a


##### 1.6. Change Types

In [18]:
df1.dtypes

store                                    int64
day_of_week                              int64
date                            datetime64[ns]
sales                                    int64
customers                                int64
open                                     int64
promo                                    int64
state_holiday                           object
school_holiday                           int64
store_type                              object
assortment                              object
competition_distance                   float64
competition_open_since_month           float64
competition_open_since_year            float64
promo2                                   int64
promo2_since_week                      float64
promo2_since_year                      float64
promo_interval                          object
month_map                               object
is_promo                                 int64
dtype: object

In [19]:
df1.competition_open_since_month = df1.competition_open_since_month.astype(int)
df1.competition_open_since_year = df1.competition_open_since_year.astype(int)

df1.promo2_since_week = df1.promo2_since_week.astype(int)
df1.promo2_since_year = df1.promo2_since_year.astype(int)

#### 